# MAC-LRN Case Study
   ## Members:
   ### Caguiat, EJ
   ### Caro, Patricia
   ### Dienzo, Jericho Rafael
 
## Dataset: "Titanic: Machine Learning from Disaster"

 
 
 
 


# Introduction of the problem and dataset
### The training dataset that we are currently using describes different features for each passenger aboard the Titanic and shows whether or not they survived. The task is to correctly classify whether or not a given person with a given set of features (from the test dataset) would survive based on the training dataset.
#### The link to the kaggle dataset that we used: https://www.kaggle.com/c/titanic

# List of requirements
### The list of requirements that we used, and what we think are necessary in order to tackle this problem, are:
#### - Pandas (A Python library built on NumPy which is commonly used for machine learning problems)
#### - NumPy (A Python library used to process numerical data)
#### - The Kaggle Dataset (specifically the Titanic dataset) which can be obtained from the link provided above
#### - Matplotlib (A Python library used for plotting)
#### - Seaborn (A Python for data visualization)

### Import of necessary libraries and data sets

In [158]:
import pandas as pd
import numpy as np
import seaborn as sns
trainfile = pd.read_csv("/Users/Jericho/Documents/Subjects/MAC-LRN/all/train.csv")
testfile = pd.read_csv("/Users/Jericho/Documents/Subjects/MAC-LRN/all/test.csv")

# Data analysis
### The dataset that we used had many different features per person, all of which we deemed necessary in determining whether or not the person would live, except for the Passenger ID, ticket field and cabin(more on this later). There were many anomalies in the dataset, such as null fields, fields that contained different datatypes than intended (int fields having chars), etc. Thus the data cleaning.

# Data Cleaning
### The removal of PassengerId was done because it had no bearing on whether or not the person would survive. 
### The removal of the Ticket field was also done for the same reasons as the PassengerId. It is also hard to identify exactly what the Ticket field is specifying (if anything at all).
### The cabin field might appear as important in determining whether or not a person lives as the other features/factors (after all, the people residing in nearest cabins to the top of the ship have a better chance of surviving), but upon closer inspection, one would come to realize that the Ticket field is nothing but an unnecessary field presenting redundant information. It presents the same data as the Fare and PClass, in that you can already derive a person's status from the Fare and PClass. We can then infer exactly what the person's social status is and, as a result, be able to infer where they are staying on the ship. 

In [159]:
columnsdropped = ['PassengerId', 'Ticket', 'Cabin']
trainfile.drop(columnsdropped, inplace = True, axis = 1)
for x in range(0, trainfile['Age'].size - 1):
    if pd.isnull(trainfile.Age.iloc[x]):
        trainfile.at[x, 'Age'] = np.random.uniform(low = trainfile.Age.min(), high = trainfile.Age.max())

print(trainfile)

     Survived  Pclass                                               Name  \
0           0       3                            Braund, Mr. Owen Harris   
1           1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2           1       3                             Heikkinen, Miss. Laina   
3           1       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4           0       3                           Allen, Mr. William Henry   
5           0       3                                   Moran, Mr. James   
6           0       1                            McCarthy, Mr. Timothy J   
7           0       3                     Palsson, Master. Gosta Leonard   
8           1       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9           1       2                Nasser, Mrs. Nicholas (Adele Achem)   
10          1       3                    Sandstrom, Miss. Marguerite Rut   
11          1       1                           Bonnell, Miss. Elizabeth   
12          

In [160]:
trainfile.fillna(np.random.uniform(low = trainfile.Age.min(), high = trainfile.Age.max()), inplace = True)